# Import

In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

import math
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pytorch_kinematics as pk
from urdf_parser_py.urdf import URDF

# Initialization

In [2]:
urdf_path = os.path.join(os.getcwd(),"..","urdf","grace.urdf")
robot = URDF.from_xml_file(urdf_path)

# URDF

In [3]:
# URDF Object Properties

for joint in robot.joints:
    print(joint.name)

torso
realsense_mount
chest_camera
neck_pitch
neck_yaw
head_pitch
eyes_pitch
lefteye_yaw
righteye_yaw


In [4]:
# Content Manipulation

for joint in robot.joints:
    if joint.name == 'realsense_mount':
        print('Joint Object:',joint.origin)
        # Getting
        print("Single Attribute:",joint.origin.position)
        # Setting
        print("Attribute Type:",type(joint.origin.position))
        joint.origin.position = [joint.origin.position[0], joint.origin.position[1], joint.origin.position[2]]
        print("Output:",joint.origin.position)

Joint Object: rpy:
- 0.0
- 0.0
- 0.0
xyz:
- 0.10749
- 0.0
- 0.44989
Single Attribute: [0.10749, 0.0, 0.44989]
Attribute Type: <class 'list'>
Output: [0.10749, 0.0, 0.44989]


In [5]:
# To XML string with filtering

temp_str = robot.to_xml_string()
words = temp_str.split()
words[5] = '>'
urdf_str = ' '.join(words)
print(urdf_str)

<?xml version="1.0" ?> <robot name="robot" > <link name="world"/> <link name="torso"> <visual> <origin xyz="0.0 0.0 0.26" rpy="0 0 0"/> <geometry> <box size="0.26 0.3 0.6"/> </geometry> <material name="robot_model"> <color rgba="0.3 0.3 0.3 0.4"/> </material> </visual> </link> <link name="realsense_mount"> <visual> <origin xyz="0.0 0.0 0.00625" rpy="0.0 0.0 0.0"/> <geometry> <box size="0.01485 0.001 0.015"/> </geometry> <material name="robot_camera"> <color rgba="0.0 0.1 0.3 0.6"/> </material> </visual> </link> <link name="realsense"> <visual> <origin xyz="-0.01485 -0.0325 0.0" rpy="0.0 0.0 0.0"/> <geometry> <box size="0.025 0.09 0.025"/> </geometry> <material name="robot_camera"> <color rgba="0.0 0.1 0.3 0.6"/> </material> </visual> </link> <link name="neck_py"/> <link name="neck"> <visual> <origin xyz="0.0 0.0 0.07" rpy="0 0 0"/> <geometry> <cylinder radius="0.045" length="0.14"/> </geometry> <material name="robot_model"> <color rgba="0.3 0.3 0.3 0.4"/> </material> </visual> </link> 

# Pytorch Kinematics

In [6]:
chain = pk.build_chain_from_urdf(urdf_str)
chain.print_tree()

world
└── torso
    ├── realsense_mount
    │   └── realsense
    └── neck_py
        └── neck
            └── head
                └── eyes
                    ├── lefteye
                    └── righteye



'world\n└── torso\n    ├── realsense_mount\n    │   └── realsense\n    └── neck_py\n        └── neck\n            └── head\n                └── eyes\n                    ├── lefteye\n                    └── righteye\n'

In [7]:
print(chain.get_joint_parameter_names())

['realsense_mount', 'neck_pitch', 'neck_yaw', 'head_pitch', 'eyes_pitch', 'lefteye_yaw', 'righteye_yaw']


In [8]:
# Specify Joint Angles (radians)
joint_cmd = {
    'realsense_mount': 0.0,
    'neck_pitch': 0.0,
    'neck_yaw': 0.0,
    'head_pitch': math.radians(10),
    'eyes_pitch': 0.0,
    'lefteye_yaw': 0.0,
    'righteye_yaw': 0.0,
}

In [9]:
# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)
ret

{'world': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'torso': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'realsense_mount': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1075, 0.0000, 0.4499]])),
 'realsense': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1223, 0.0325, 0.4774]])),
 'neck_py': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'neck': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'head': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[-0.0177,  0.0000,  0.6660]])),
 'eyes': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[0.0570, 0.0000, 0.7318]])),
 'lefteye': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[0.0570, 0.0290, 0.7318]])),
 'righteye': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tens

In [10]:
# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)
ret

{'world': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'torso': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'realsense_mount': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1075, 0.0000, 0.4499]])),
 'realsense': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1223, 0.0325, 0.4774]])),
 'neck_py': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'neck': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'head': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[-0.0177,  0.0000,  0.6660]])),
 'eyes': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[0.0570, 0.0000, 0.7318]])),
 'lefteye': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[0.0570, 0.0290, 0.7318]])),
 'righteye': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tens

In [11]:
ret['lefteye'].get_matrix()

tensor([[[ 0.9848,  0.0000, -0.1736,  0.0570],
         [ 0.0000,  1.0000,  0.0000,  0.0290],
         [ 0.1736,  0.0000,  0.9848,  0.7318],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])

In [12]:
ret['righteye'].get_matrix()

tensor([[[ 0.9848,  0.0000, -0.1736,  0.0570],
         [ 0.0000,  1.0000,  0.0000, -0.0290],
         [ 0.1736,  0.0000,  0.9848,  0.7318],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])

# Eye Projection

In [13]:
import torch
import cv2

## Function Prototype

In [14]:
#--- Input
pts_3d = torch.tensor([[1.0, 2.0, 5.0],
                      [2.0, 3.0, 8.0],
                      [3.0, 1.0, 10.0]])
cam_mtx = torch.tensor([[1000.0, 0, 320.0],
                        [0, 1000.0 , 240.0],
                        [0, 0, 1.0]])
ext_mtx = torch.tensor([[1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 1.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
dist_coef = (0.1, -0.2, 0.01, 0.01, 0.0)

In [15]:
# Transform to Homogeneous Coordinates
pts_3d_homo = torch.cat((pts_3d, torch.ones(pts_3d.shape[0], 1)), dim=1)
print(pts_3d_homo)
print(pts_3d_homo.T)

tensor([[ 1.,  2.,  5.,  1.],
        [ 2.,  3.,  8.,  1.],
        [ 3.,  1., 10.,  1.]])
tensor([[ 1.,  2.,  3.],
        [ 2.,  3.,  1.],
        [ 5.,  8., 10.],
        [ 1.,  1.,  1.]])


In [16]:
# Apply extrinsic transformation (rotation + translation)
pts_3d_transformed = ext_mtx @ pts_3d_homo.T
pts_3d_transformed

tensor([[ 1.,  2.,  3.],
        [ 2.,  3.,  1.],
        [ 5.,  8., 10.],
        [ 1.,  1.,  1.]])

In [17]:
# Convert back from homogeneous to 2D coordinates
pts_2d_prime = pts_3d_transformed[:2, :] / pts_3d_transformed[2, :]
x_prime = pts_2d_prime[0]
y_prime = pts_2d_prime[1]
print(pts_2d_prime)
print(x_prime)
print(y_prime)

tensor([[0.2000, 0.2500, 0.3000],
        [0.4000, 0.3750, 0.1000]])
tensor([0.2000, 0.2500, 0.3000])
tensor([0.4000, 0.3750, 0.1000])


In [18]:
# Distortion Parameters Calculation
k1, k2, p1, p2, k3 = dist_coef

r2 = x_prime**2 + y_prime**2
r4 = r2**2
r6 = r2*r4
a1 = 2*x_prime*y_prime
a2 = r2 + 2*x_prime**2
a3 = r2 + 2*y_prime**2

# Radial distortion
radial_distortion = 1 + k1*r2 + k2*r4 + k3*r6

# Tangential distortion
tangential_distortion_x = p1*a1 + p2*a2
tangential_distortion_y = p1*a3 + p2*a1

# Apply distortion
x_double_prime = x_prime*radial_distortion + tangential_distortion_x
y_double_prime = y_prime* radial_distortion + tangential_distortion_y

print(x_double_prime)
print(y_double_prime)

tensor([0.2068, 0.2582, 0.3058])
tensor([0.4116, 0.3862, 0.1026])


In [19]:
# Intrinsic Camera Matrix
fx = cam_mtx[0,0]
cx = cam_mtx[0,2]
fy = cam_mtx[1,1]
cy = cam_mtx[1,2]

u = x_double_prime*fx + cx
v = y_double_prime*fy + cy

px_pts = torch.stack((u, v), dim=1)

print(u)
print(v)
print(px_pts)

tensor([526.8000, 578.1714, 625.8000])
tensor([651.6000, 626.2415, 342.6000])
tensor([[526.8000, 651.6000],
        [578.1714, 626.2415],
        [625.8000, 342.6000]])


## Function

In [20]:
def projectPoints(pts_3d, cam_mtx, ext_mtx, dist_coef):
    # Transform to Homogeneous Coordinates
    pts_3d_homo = torch.cat((pts_3d, torch.ones(pts_3d.shape[0], 1)), dim=1)
    
    # Apply extrinsic transformation (rotation + translation)
    pts_3d_transformed = ext_mtx @ pts_3d_homo.T
    
    # Convert back from homogeneous to 2D coordinates
    pts_2d_prime = pts_3d_transformed[:2, :] / pts_3d_transformed[2, :]
    x_prime = pts_2d_prime[0]
    y_prime = pts_2d_prime[1]
    
    # Distortion Parameters Calculation
    k1, k2, p1, p2, k3 = dist_coef
    
    r2 = x_prime**2 + y_prime**2
    r4 = r2**2
    r6 = r2*r4
    a1 = 2*x_prime*y_prime
    a2 = r2 + 2*x_prime**2
    a3 = r2 + 2*y_prime**2
    
    # Radial distortion
    radial_distortion = 1 + k1*r2 + k2*r4 + k3*r6
    
    # Tangential distortion
    tangential_distortion_x = p1*a1 + p2*a2
    tangential_distortion_y = p1*a3 + p2*a1
    
    # Apply distortion
    x_double_prime = x_prime*radial_distortion + tangential_distortion_x
    y_double_prime = y_prime* radial_distortion + tangential_distortion_y
    
    # Intrinsic Camera Matrix
    fx = cam_mtx[0,0]
    cx = cam_mtx[0,2]
    fy = cam_mtx[1,1]
    cy = cam_mtx[1,2]
    
    u = x_double_prime*fx + cx
    v = y_double_prime*fy + cy
    
    px_pts = torch.stack((u, v), dim=1)

    return px_pts

In [21]:
# Testing the Function

#--- Input
pts_3d = torch.tensor([[1.0, 2.0, 5.0],
                      [2.0, 3.0, 8.0],
                      [3.0, 1.0, 10.0]])
cam_mtx = torch.tensor([[1000.0, 0, 320.0],
                        [0, 1000.0 , 240.0],
                        [0, 0, 1.0]])
ext_mtx = torch.tensor([[1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 1.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
dist_coef = (0.1, -0.2, 0.01, 0.01, 0.0)

#--- Function
pytorch_pts = projectPoints(pts_3d, cam_mtx, ext_mtx, dist_coef)
pytorch_pts

tensor([[526.8000, 651.6000],
        [578.1714, 626.2415],
        [625.8000, 342.6000]])

## OpenCV Comparison

In [22]:
# Input

#--- Input
pts_3d = np.array([[1.0, 2.0, 5.0],
                      [2.0, 3.0, 8.0],
                      [3.0, 1.0, 10.0]])
cam_mtx = np.array([[1000.0, 0, 320.0],
                        [0, 1000.0 , 240.0],
                        [0, 0, 1.0]])
rvec = np.array([0.0,0.0,0.0]).reshape(-1,1)
tvec = np.array([0.0,0.0,0.0]).reshape(-1,1)
dist_coef = np.array([0.1, -0.2, 0.01, 0.01, 0.0])

In [23]:
# OpenCV Function

opencv_pts,_ = cv2.projectPoints(pts_3d, rvec, tvec, cam_mtx, dist_coef)
opencv_pts

array([[[526.8       , 651.6       ]],

       [[578.17138672, 626.24145508]],

       [[625.8       , 342.6       ]]])

In [24]:
# Comparison

opencv_pts.squeeze() - pytorch_pts.numpy()

array([[ 1.22070312e-05,  2.44140625e-05],
       [ 0.00000000e+00,  0.00000000e+00],
       [-4.88281249e-05, -6.10351560e-06]])